# Lab Assignment One: Exploring Table Data

#### Everett Cienkus, Blake Miller, Colin Weil

## 1. Business Understanding
This dataset contains data for all NBA games from the 2004-2005 season until March of the 2021-2022 season.
From the description of the dataset, the data was collected from the NBA stats website.

This dataset can be useful for various parties, including NBA teams, coaches, players, sports bettors, or  anyone looking to better understand basketball statistics. In our exploration we will be specifically looking at the application of this data for an NBA team GM. According to www.investopedia.com, of the 30 NBA teams "a team is worth on average $2.12 billion", and "total revenue across the organization reached 8.76 billion in the 2018-2019 season."

## 2. Data Understanding

### 2.1 Data Description

In [58]:
import pandas as pd
import numpy as np

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)
df = pd.read_csv('NBA_datasets/games.csv') # read in the csv file
#Split stats into home and away
home_df = df[["GAME_DATE_EST","PTS_home","FG_PCT_home","FT_PCT_home","FG3_PCT_home","AST_home","REB_home", "HOME_TEAM_WINS"]]
away_df = df[["GAME_DATE_EST","PTS_away","FG_PCT_away","FT_PCT_away","FG3_PCT_away","AST_away","REB_away", "HOME_TEAM_WINS"]]

#Convert HOME_TEAM_WINS the to appropriate value based on home or away team
home_df=home_df.rename(columns={'HOME_TEAM_WINS':'WIN'})
away_df=away_df.rename(columns={'HOME_TEAM_WINS':'WIN'})
away_df["WIN"]=np.logical_not(away_df["WIN"])
home_df["WIN"]=home_df["WIN"].astype(bool)

home_df.columns = ['GAME_DATE_EST', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'WIN']
home_df["HOME"]= 1
away_df.columns = ['GAME_DATE_EST', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'WIN']
away_df["HOME"]= 0
df = pd.concat([home_df, away_df])

#display
display(df.head(51592))

Pandas: 1.4.2
Numpy: 1.21.5
25796
25796
51592


,GAME_DATE_EST,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,WIN,HOME
0,2022-03-12,104.0,0.398,0.760,0.333,23.0,53.0,False,1
1,2022-03-12,101.0,0.443,0.933,0.429,20.0,46.0,True,1
2,2022-03-12,108.0,0.412,0.813,0.324,28.0,52.0,False,1
3,2022-03-12,122.0,0.484,0.933,0.400,33.0,55.0,True,1
4,2022-03-12,115.0,0.551,0.750,0.407,32.0,39.0,False,1
...,...,...,...,...,...,...,...,...,...
25791,2014-10-06,87.0,0.366,0.643,0.375,17.0,43.0,False,0
25792,2014-10-06,85.0,0.411,0.636,0.267,17.0,47.0,True,0
25793,2014-10-06,95.0,0.387,0.659,0.500,19.0,43.0,False,0
25794,2014-10-05,94.0,0.469,0.725,0.385,18.0,45.0,False,0


(NEED TO DO) Need to describe the changes made

In [29]:
# find the data type
print(home_df.info())
print()
print(away_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GAME_DATE_EST  25796 non-null  object 
 1   PTS_home       25697 non-null  float64
 2   FG_PCT_home    25697 non-null  float64
 3   FT_PCT_home    25697 non-null  float64
 4   FG3_PCT_home   25697 non-null  float64
 5   AST_home       25697 non-null  float64
 6   REB_home       25697 non-null  float64
 7   WIN            25796 non-null  bool   
dtypes: bool(1), float64(6), object(1)
memory usage: 1.4+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25796 entries, 0 to 25795
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GAME_DATE_EST  25796 non-null  object 
 1   PTS_away       25697 non-null  float64
 2   FG_PCT_away    25697 non-null  float64
 3   FT_PCT_away    25697 non-null  float64
 4   FG3_PCT_

(NEED TO DO) Describe the data

In [31]:
# find the data summary
display(home_df.describe())
display(away_df.describe())

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home
count,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000
mean,103.106044,0.460313,0.759705,0.355896,22.736779,43.345799
std,13.174726,0.056629,0.100692,0.111940,5.177566,6.621832
min,36.000000,0.250000,0.143000,0.000000,6.000000,15.000000
25%,94.000000,0.421000,0.696000,0.286000,19.000000,39.000000
50%,103.000000,0.459000,0.765000,0.355000,23.000000,43.000000
75%,112.000000,0.500000,0.829000,0.429000,26.000000,48.000000
max,168.000000,0.684000,1.000000,1.000000,50.000000,72.000000


,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away
count,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000,25697.000000
mean,100.294120,0.449265,0.758082,0.349413,21.403899,42.085146
std,13.343016,0.055528,0.103418,0.110194,5.140897,6.526951
min,33.000000,0.244000,0.143000,0.000000,4.000000,19.000000
25%,91.000000,0.412000,0.692000,0.278000,18.000000,38.000000
50%,100.000000,0.448000,0.765000,0.350000,21.000000,42.000000
75%,109.000000,0.487000,0.833000,0.420000,25.000000,46.000000
max,168.000000,0.687000,1.000000,1.000000,46.000000,81.000000


### 2.2 Data Quality

In [ ]:
# Code goes here

## 3. Data Visualization

### 3.1 Data Exploration

In [ ]:
# Code goes here

### 3.2 Data Relationship Exploration

In [ ]:
# Code goes here

## 4. Dimensionality Reduction

In [ ]:
# Code goes here

In [19]:
# Code goes here